In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import spikeinterface.full as si
import spikeinterface.sortingcomponents.peak_detection as pkd
import numpy as np
import time

/home/aurelien/Documents/Rust/spikeinterface_rust/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
job_kwargs={"n_jobs": -1, "mp_context": "fork", "pool_engine":"thread", "progress_bar": True, "chunk_duration":"1s"}

In [4]:
rec, sorting = si.generate_ground_truth_recording(durations=[60],num_channels=128,sampling_frequency=25000.0,num_units= 100, seed=42)
rec = rec.save_to_memory(**job_kwargs)
_ = pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs=job_kwargs)

noise_level (workers: 20 threads): 100%|██████████| 20/20 [00:00<00:00, 214.32it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 206.26it/s]


In [ ]:
peaks_numba = pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs=job_kwargs)
peaks_rust = pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "rust"}, job_kwargs=job_kwargs)

print(len(peaks_numba))
print(len(peaks_rust))

detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 230.68it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 695.44it/s]

141371
141519


In [6]:
np.setdiff1d(peaks_rust, peaks_numba)

array([(    553,  30,  30.76909447, 0), (  13191,  45,  29.40257835, 0),
       (  24510,  77,  31.68558311, 0), (  24787,  19,  34.81524658, 0),
       (  35641, 110,  29.64479065, 0), (  49782,  31,  31.54408264, 0),
       (  49989,  77,  30.34431267, 0), (  70699,   9,  29.41254616, 0),
       (  74917,  99, -37.89614487, 0), (  85201,   0, -27.57130814, 0),
       (  92080,  36,  29.131464  , 0), ( 107853,  47,  29.4995842 , 0),
       ( 124932,  48,  31.55229759, 0), ( 127822,  24,  30.6662941 , 0),
       ( 149790,  15,  30.68748474, 0), ( 149902, 125,  32.64838409, 0),
       ( 149903, 125,  29.81681442, 0), ( 150828, 101,  28.73488617, 0),
       ( 171753, 122,  28.3742218 , 0), ( 174753,  94,  32.35191727, 0),
       ( 174834,   6,  33.94483948, 0), ( 174980,  81,  35.29873657, 0),
       ( 195552,   4,  28.90981483, 0), ( 199620,  88,  30.90521622, 0),
       ( 199953,   0, -45.89426041, 0), ( 199954,   0, -41.4924469 , 0),
       ( 224718,  45,  34.18171692, 0), ( 248514,  

In [7]:
%timeit pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "rust"}, job_kwargs=job_kwargs)

detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 958.03it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 1028.95it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 1069.15it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 1092.24it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 1187.61it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 1073.45it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 1038.71it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 1011.44it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 1117.04it/s]
detect peaks (locally_exclusive) (workers: 24 t

59.5 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%timeit pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs=job_kwargs)

detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 687.62it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 701.61it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 685.43it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 679.23it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 751.01it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 687.94it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 696.81it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 719.60it/s]
detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 747.69it/s]
detect peaks (locally_exclusive) (workers: 24 threads):

KeyboardInterrupt: 

In [ ]:
for i in range(1,4):
    job_kwargs={"n_jobs": -1, "mp_context": "fork", "pool_engine":"thread", "progress_bar": False, "chunk_duration": str(500*i) +"ms"}
    print("Chunk duration :" + str(500*i) +" ms")
    %timeit pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "rust"}, job_kwargs=job_kwargs)

Chunk duration :500 ms
72.7 ms ± 496 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :1000 ms
59.9 ms ± 821 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :1500 ms
56.3 ms ± 810 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
for i in range(1,4):
    job_kwargs={"n_jobs": -1, "mp_context": "fork", "pool_engine":"thread", "progress_bar": False, "chunk_duration": str(500*i) +"ms"}
    print("Chunk duration :" + str(500*i) +" ms")
    %timeit pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs=job_kwargs)

Chunk duration :500 ms
81.4 ms ± 928 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :1000 ms
87 ms ± 647 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :1500 ms
108 ms ± 1.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
